In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorboard import summary as summary_lib
# from tensorboard import plugins as plugins_lib
# from tensorboard.plugins.pr_curve import summary

import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline




In [2]:
data_dir_1 = '/Users/derekwang/Work/Stock_out/Data/STOCK_OUT_master_training.csv'
data_dir_2 = '/Users/derekwang/Work/Stock_out/Data/STOCK_OUT_master_training_0219.csv'

In [3]:
data_raw = pd.read_csv(data_dir_1, sep=',', header=0)
# data_raw['TRUCK_CONSTRAINT'] = data_raw.TRUCK_CONSTRAINT.astype('int64')
# data_raw_2 = pd.read_csv(data_dir_2, sep=',', header=0)


data_clean = data_raw # outlier_remove(data_raw)
data_clean.head()

,LOC_ID,ITEM_ID,CAL_DT,STR_INV_STR_VOL_LVL_CD,STR_INV_STRD_VOL_LVL_CD,STR_INV_STRC_VOL_LVL_CD,STR_INV_STRSC_VOL_LVL_CD,ITEM_VLCTY_CD,OH_QTY,OO_QTY,...,REV_TM_DAYS_CNT,SFTY_STK_QTY,SVC_LVL_VAL,ASL_OSL_VAL,ADJ_ASW_QTY,CAR_PARM_CURR_RETL_AMT,WK1_FCST,WK2_FCST,WK3_FCST,SIMPL_OOS_IND
0,2682,6040770,2018-01-21,2,1,1,2,2,0.0,15,...,7,18.351539,99.70,77.08,4.490,13.27,5.155,3.969,3.513,1
1,359,5983727,2018-01-21,2,1,2,2,3,11.0,0,...,7,12.731372,95.00,98.77,2.890,1.98,5.066,5.114,5.018,1
2,1262,719971,2018-01-21,2,1,2,2,1,13.0,90,...,4,53.837502,88.68,97.72,44.697,3.48,53.784,52.318,50.330,1
3,1366,2601810,2018-01-21,2,1,1,2,2,40.0,0,...,7,25.809154,99.30,99.90,12.470,0.96,12.796,12.548,11.958,0
4,755,5218758,2018-01-21,1,1,1,1,1,121.0,30,...,7,63.625968,99.80,100.00,25.311,19.98,29.909,29.334,28.183,0


In [4]:
numerical_columns = []
for x in data_clean.columns[3:-1]:
    numerical_columns.append(x)
        
# numerical_columns

In [5]:
def feats_norm(df, numerical_columns):
    norm_df = df.copy()
    normalizer = pd.DataFrame(columns = numerical_columns, index = ['mean', 'std'])
    for col in numerical_columns:
        mean = norm_df[col].mean()
        std = norm_df[col].std()
        normalizer.loc['mean', col] = mean
        normalizer.loc['std', col] = std
        norm_df[col] = (norm_df[col] - mean) / (std + 10.0** -6)
    return norm_df, normalizer

def feats_norm_with_normalizer(df, normalizer, numerical_columns):
    norm_df = df.copy()
    for col in numerical_columns:
        mean = normalizer.loc['mean', col]
        std = normalizer.loc['std', col]
        norm_df[col] = (norm_df[col] - mean) / (std + 10.0** -6)
    return norm_df

data_norm, normalizer = feats_norm(data_clean, numerical_columns)
data_norm.describe()   

,LOC_ID,ITEM_ID,STR_INV_STR_VOL_LVL_CD,STR_INV_STRD_VOL_LVL_CD,STR_INV_STRC_VOL_LVL_CD,STR_INV_STRSC_VOL_LVL_CD,ITEM_VLCTY_CD,OH_QTY,OO_QTY,SELL_UOM_QTY,...,REV_TM_DAYS_CNT,SFTY_STK_QTY,SVC_LVL_VAL,ASL_OSL_VAL,ADJ_ASW_QTY,CAR_PARM_CURR_RETL_AMT,WK1_FCST,WK2_FCST,WK3_FCST,SIMPL_OOS_IND
count,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3380064.0,...,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06,3.380064e+06
mean,1.448084e+03,2.157870e+06,-7.453410e-17,2.771485e-16,-3.640601e-16,-2.436485e-16,-1.348072e-16,2.183553e-16,1.988641e-18,0.0,...,1.192176e-16,-1.431149e-16,-7.163481e-16,-1.075960e-15,5.858188e-18,7.863751e-17,-4.972286e-17,5.136754e-17,7.104242e-18,1.836152e-01
std,9.161126e+02,2.296992e+06,9.999992e-01,9.999992e-01,9.999992e-01,9.999992e-01,9.999990e-01,1.000000e+00,9.999999e-01,0.0,...,9.999997e-01,9.999999e-01,9.999998e-01,9.999999e-01,1.000000e+00,1.000000e+00,9.999999e-01,9.999999e-01,9.999999e-01,3.871701e-01
min,1.700000e+01,4.100000e+01,-2.251790e+00,-2.298043e+00,-2.370991e+00,-2.436508e+00,-1.941136e+00,-3.072403e-01,-6.562090e-02,0.0,...,-1.425028e+00,-3.148387e-01,-7.845910e+00,-6.375849e+00,-9.937052e-02,-2.854842e-01,-1.183186e-01,-1.116778e-01,-1.089195e-01,0.000000e+00
25%,7.700000e+02,4.015770e+05,-7.304900e-01,-7.732060e-01,-8.374773e-01,-8.971896e-01,-9.695040e-01,-2.284443e-01,-6.562090e-02,0.0,...,-5.342902e-01,-2.688153e-01,-1.414377e-01,3.047784e-01,-8.737522e-02,-2.425251e-01,-1.055900e-01,-9.924547e-02,-9.690676e-02,0.000000e+00
50%,1.296000e+03,7.872560e+05,3.015987e-02,-1.078734e-02,-7.072041e-02,-1.275301e-01,2.128064e-03,-1.693472e-01,-6.562090e-02,0.0,...,3.564475e-01,-1.856167e-01,4.577990e-01,3.047784e-01,-7.403046e-02,-1.979617e-01,-8.714378e-02,-8.163303e-02,-7.972796e-02,0.000000e+00
75%,2.077000e+03,3.810282e+06,7.908097e-01,7.516313e-01,6.960365e-01,6.421293e-01,9.737602e-01,-3.145416e-02,-6.562090e-02,0.0,...,3.564475e-01,-2.476334e-02,5.690859e-01,3.047784e-01,-3.624527e-02,-6.427163e-02,-4.140253e-02,-3.818468e-02,-3.709218e-02,0.000000e+00
max,2.627600e+04,7.327038e+06,1.551460e+00,1.514050e+00,1.462793e+00,1.411789e+00,1.945392e+00,2.420891e+02,4.994522e+02,0.0,...,7.185437e+00,2.981881e+02,6.803727e-01,3.047784e-01,7.166816e+02,7.097001e+01,5.615079e+02,6.388136e+02,6.251030e+02,1.000000e+00


In [6]:
def rand_datasets(df, train_pct):
    rand_num = np.random.rand(len(df))
    
    df_train = df[rand_num < train_pct]
    df_cross = df[rand_num >= train_pct]
    
    return df_train, df_cross

train, test = rand_datasets(data_norm, 0.80)

train_x, train_y = train.iloc[:,3:-1], train.pop('SIMPL_OOS_IND')
test_x, test_y = test.iloc[:,3:-1], test.pop('SIMPL_OOS_IND')

In [7]:
def model_fn(features, labels, mode, params):
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)
        
    logits = tf.layers.dense(net, params['n_classes'], activation=None)
    
    predicted_classes = tf.argmax(logits, 1)
    predicted_probabilities = tf.nn.softmax(logits)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    l2_regularizer = tf.contrib.layers.l2_regularizer(scale = params['regularization_rate'])
    l2_penalty = tf.contrib.layers.apply_regularization(
                                                    regularizer = l2_regularizer,
                                                    weights_list = tf.trainable_variables())
    
    loss = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels, logits=logits)
        )
    
    regularized_loss = loss + l2_penalty
    
    
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('regularized_loss', regularized_loss)
    
    accuracy = tf.metrics.accuracy(labels = labels,
                                   predictions=predicted_classes,
                                  name='acc_op')
    
    precision = tf.contrib.metrics.streaming_precision(predictions = predicted_classes, 
                                                       labels = labels)
    
    recall = tf.contrib.metrics.streaming_recall(predictions = predicted_classes, 
                                                       labels = labels)
    
    pr_auc = tf.contrib.metrics.streaming_auc(predictions=predicted_probabilities[:,1], 
                                             labels = labels,
                                             curve = 'PR',
                                             num_thresholds = 1000)
    

    pr_curve = summary_lib.pr_curve_streaming_op(
              name="pr_curve",
              labels=labels,
              predictions=predicted_probabilities[:,1]
      )
    


    
    metrics = {'accuracy': accuracy,
               'precision': precision,
               'recall': recall,
               'pr_auc': pr_auc,
#                "pr_curve": pr_curve
              }
    
    tf.summary.scalar('accuracy', accuracy[1])
    tf.summary.scalar('precision', precision[1])
    tf.summary.scalar('recall', recall[1])
    tf.summary.scalar('pr_auc', pr_auc[1])
    
    

    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss= regularized_loss, eval_metric_ops=metrics)
    
    assert mode == tf.estimator.ModeKeys.TRAIN
    
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(loss=regularized_loss, global_step = tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=regularized_loss, train_op=train_op)


In [8]:
def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

def eval_input_fn(features, labels, batch_size):
    features=dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)
    
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset

In [9]:
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))


In [10]:
model_dir = '/Users/derekwang/Work/Stock_out/model_tunning_'

In [11]:
node_dic = {'1': 200,
            '2': 400,
            '3': 600,
            '4': 800,
            '5': 1000,

            }

In [13]:
%%time
#final tunned model


for key, value in node_dic.items():
    classifier = tf.estimator.Estimator(
    model_fn = model_fn, 
    model_dir = model_dir + key,
    params={
        'feature_columns': my_feature_columns,
        'hidden_units': [512, 256, 16],
        'n_classes': 2,
        'learning_rate': 1e-5,
        'regularization_rate': 2e-3
    })
    
    classifier.train(
            input_fn=lambda:train_input_fn(train_x, train_y, value),
            steps = 7e4)
    
    eval_result = classifier.evaluate(
        input_fn =lambda:eval_input_fn(test_x, test_y, 200))

    print('Test set {} accuracy: {accuracy:0.3f}\n'.format(key, **eval_result))
    print('Test set {} precision: {precision:0.3f}\n'.format(key, **eval_result))
    print('Test set {} recall: {recall:0.3f}\n'.format(key, **eval_result))
    print('Test set {} pr_auc: {pr_auc:0.3f}\n'.format(key, **eval_result))    

Instructions for updating:
Please switch to tf.metrics.auc. Note that the order of the labels and predictions arguments has been switched.


Test set 1 accuracy: 0.829

Test set 1 precision: 0.701

Test set 1 recall: 0.116

Test set 1 pr_auc: 0.387

Test set 2 accuracy: 0.829

Test set 2 precision: 0.738

Test set 2 recall: 0.107

Test set 2 pr_auc: 0.409

Test set 3 accuracy: 0.829

Test set 3 precision: 0.682

Test set 3 recall: 0.123

Test set 3 pr_auc: 0.409

Test set 4 accuracy: 0.829

Test set 4 precision: 0.690

Test set 4 recall: 0.123

Test set 4 pr_auc: 0.412

Test set 5 accuracy: 0.829

Test set 5 precision: 0.703

Test set 5 recall: 0.117

Test set 5 pr_auc: 0.410

CPU times: user 5h 53min 26s, sys: 12min 42s, total: 6h 6min 8s
Wall time: 2h 46min 18s
